In [1]:
from langchain.graphs import Neo4jGraph
from langchain_openai import AzureChatOpenAI
from dotenv import load_dotenv
import os
from langchain.prompts import PromptTemplate
from langchain.chains import GraphCypherQAChain

load_dotenv()

NEO4J_URI = "neo4j://localhost:7687"
NEO4J_USER = "neo4j"
NEO4J_PASSWORD = "4NROB-HI"

graph = Neo4jGraph(
    url=NEO4J_URI,
    username=NEO4J_USER,
    password=NEO4J_PASSWORD,
)

AZURE_API_KEY     = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_ENDPOINT    = os.getenv("AI_BASE")
AZURE_DEPLOYMENT  = os.getenv("AZURE_DEPLOYMENT")
AZURE_API_VERSION = os.getenv("AZURE_OPENAI_VERSION")


llm = AzureChatOpenAI(
    api_key=AZURE_API_KEY,
    azure_endpoint=AZURE_ENDPOINT,
    deployment_name=AZURE_DEPLOYMENT,
    api_version=AZURE_API_VERSION,
    temperature=0,
    # max_tokens=2000,
)


/tmp/ipykernel_21025/1923221998.py:14: LangChainDeprecationWarning: The class `Neo4jGraph` was deprecated in LangChain 0.3.8 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-neo4j package and should be used instead. To use it run `pip install -U :class:`~langchain-neo4j` and import as `from :class:`~langchain_neo4j import Neo4jGraph``.
  graph = Neo4jGraph(


In [2]:
schema_str = graph.schema

In [3]:
import pprint
pprint.pprint(schema_str)

('Node properties:\n'
 'InventoryLevel1 {name: STRING}\n'
 'InventoryLevel2 {name: STRING}\n'
 'InventoryLevel3 {name: STRING}\n'
 'InventoryLevel4 {name: STRING}\n'
 'GroupCompany {name: STRING}\n'
 'Relationship properties:\n'
 'HAS_AMOUNT {year: STRING, amount: FLOAT}\n'
 'The relationships:\n'
 '(:InventoryLevel1)-[:HAS_SUB_INVENTORY]->(:InventoryLevel2)\n'
 '(:InventoryLevel2)-[:HAS_SUB_INVENTORY]->(:InventoryLevel3)\n'
 '(:InventoryLevel2)-[:HAS_SUB_INVENTORY]->(:InventoryLevel4)\n'
 '(:InventoryLevel3)-[:HAS_SUB_INVENTORY]->(:InventoryLevel4)\n'
 '(:GroupCompany)-[:HAS_AMOUNT]->(:InventoryLevel3)\n'
 '(:GroupCompany)-[:HAS_AMOUNT]->(:InventoryLevel4)\n'
 '(:GroupCompany)-[:HAS_AMOUNT]->(:InventoryLevel2)\n'
 '(:GroupCompany)-[:HAS_AMOUNT]->(:InventoryLevel1)')


In [4]:
def extract_nodes_by_label_from_markdown(md_path):
    with open(md_path, "r", encoding="utf-8") as f:
        md = f.read()
    
    import re

    # Patterns to extract each section (greedy until next heading or end of string)
    def extract_section(title):
        pattern = rf"### {re.escape(title)}\n((?:- .+\n)+)"
        match = re.search(pattern, md)
        if match:
            # Remove the leading "- " from each line and join
            items = [line[2:].strip() for line in match.group(1).strip().split("\n") if line.startswith("- ")]
            return "\n".join(items)
        else:
            return ""

    group_companies = extract_section("GroupCompany")
    inventorylevel1 = extract_section("InventoryLevel1")
    inventorylevel2 = extract_section("InventoryLevel2")
    inventorylevel3 = extract_section("InventoryLevel3")
    inventorylevel4 = extract_section("InventoryLevel4")

    return group_companies, inventorylevel1, inventorylevel2, inventorylevel3, inventorylevel4

# Example usage:
md_path = "kg_schema_overview.md"
group_companies, inventorylevel1, inventorylevel2, inventorylevel3, inventorylevel4 = extract_nodes_by_label_from_markdown(md_path)

# print("### GroupCompany")
# print(group_companies)
# print("\n### InventoryLevel1")
# print(inventorylevel1)
# print("\n### InventoryLevel2")
# print(inventorylevel2)
# print("\n### InventoryLevel3")
# print(inventorylevel3)
# print("\n### InventoryLevel4")
# print(inventorylevel4)


In [5]:
inventorylevel2

'Finished Goods\nInventory Machinery Spares\nOther Inventory\nRaw Material & Mfg. Componets\nStock in Trade\nStores and Spares\nTools\nWork-in-Progress'

In [6]:
inventory_type = inventorylevel1 + '\n' + inventorylevel2 + '\n' + inventorylevel3 + '\n' + inventorylevel4

In [7]:
inventory_type

'Inventories\nFinished Goods\nInventory Machinery Spares\nOther Inventory\nRaw Material & Mfg. Componets\nStock in Trade\nStores and Spares\nTools\nWork-in-Progress\nFinished Goods\nRaw Material & Mfg. Componet\nStock In Transit - Finised Goods\nStock In Transit - Raw Material\nStock In Transit - Traded Goods\nStock in Trade\nClg Stock - Construction W I P - B/s\nClg Stock - Finished Goods - B/s\nClg Stock - Mfg Component B/s\nClg Stock - Raw Material-B/s\nClg Stock - Wip - B/s\nClg Stock In Transit - FG B/s\nClg Stock In Transit - Rm - B/s\nClosing Stock- Gunny Bags\nConstruction Work In Progress\nFood & Beverages\nG-Stock Materiali Ricoperti CBN\nIn Transit - Stores & Spares\nInd-AS  Inventories\nInventorie Consumable Stores & Misc.\nInventory Dies\nInventory Machinery Spares-Imported\nInventory Manufactured Tools\nInventory Oils, Grease & Chemicals\nInventory Packing Material\nInventory Spares\nInventory Spares- Manual Adj\nInventory Stores- Manual Adj\nInventory Tools\nInventory To

In [8]:
group_companies

'AS\nAgri\nCE\nCONT SOUR\nCORP\nFES\nHO\nINTNL OPER\nMDS\nNON MRV\nPowerol\nSBU\nSOURCING\nSWARAJ\nSynergy\nTPDS\nTW'

In [9]:
NORMALIZE_PROMPT = """
You are a strict information extraction agent.
Your task is to extract and map relevant keywords from the user query to their exact forms as found in the provided lists.
You MUST return only the following JSON object and nothing else. Do not add explanations, notes, or preamble.

Group company names:
{group_companies}

Inventory type names:
{inventory_type}

Year values:
- FY23
- FY24

User query:
{user_query}

Return only this JSON object (use null if not found):
{{
  "company_name": "<matching group company name>",
  "inventory_type": "<matching inventory type>",
  "year": "<matching year>",
  "canonicalized_query": "<original query with all keywords replaced by their canonical KG forms>"
}}
"""


In [10]:
import json
import re
from pydantic import BaseModel, ValidationError
from typing import Optional

class NormalizationResult(BaseModel):
    company_name: Optional[str]
    inventory_type: Optional[str]
    year: Optional[str]
    canonicalized_query: Optional[str]

def normalize_entities(user_query):
    prompt = NORMALIZE_PROMPT.format(user_query=user_query, group_companies=group_companies, inventory_type=inventory_type)
    result = llm.invoke(prompt)
    content = result.content if hasattr(result, 'content') else str(result)
    print("Raw LLM output:", content)
    # Extract JSON robustly
    match = re.search(r"\{.*\}", content, re.DOTALL)
    json_str = match.group(0) if match else "{}"
    try:
        normalized = NormalizationResult.model_validate_json(json_str)
    except ValidationError as ve:
        print("Validation error:", ve)
        # Optionally, try to coerce to dict
        normalized = None
    return normalized

# Example usage:
user_query = "what are the value of Seed Potato Facility of the AS sector for year 2023?"
normalized = normalize_entities(user_query)
print("Normalized:", normalized)
if normalized:
    print("Canonicalized Query:", normalized.canonicalized_query)


Raw LLM output: {
  "company_name": "AS",
  "inventory_type": "WIP- Seed Potato Facility A/c",
  "year": "FY23",
  "canonicalized_query": "what are the value of WIP- Seed Potato Facility A/c of the AS sector for year FY23?"
}
Normalized: company_name='AS' inventory_type='WIP- Seed Potato Facility A/c' year='FY23' canonicalized_query='what are the value of WIP- Seed Potato Facility A/c of the AS sector for year FY23?'
Canonicalized Query: what are the value of WIP- Seed Potato Facility A/c of the AS sector for year FY23?


In [11]:
CYPHER_GENERATION_TEMPLATE = """
Task: Generate one or more Cypher queries that extract all first-level connections—nodes, relationships, and every property on them—for the inventories and/or group companies mentioned in the user’s question, using the schema below.

Instructions:

1. Node labels allowed: GroupCompany, InventoryLevel1, InventoryLevel2, InventoryLevel3, InventoryLevel4
2. Relationship types allowed: HAS_AMOUNT, HAS_SUB_INVENTORY
3. Extract all possible nodes that are directly related to the target inventories or group companies specified in the question.
4. Fetch every possible relationship (edge) of the allowed types connecting those nodes.
5. Always return every property on every matched node and relationship (e.g., name, year, amount, id).
6. If the question names a group company, inventory, or year, include an explicit WHERE clause filter for it.
7. Explicitly name relationship variables in MATCH (e.g., r1, r2) and use those variables in RETURN.
8. Do not aggregate, create, or assume any properties, labels, or relationship types that aren’t in the schema.
9. When the question mentions a single inventory hierarchy (e.g., “Stores and Spares”), you may traverse that hierarchy with HAS_SUB_INVENTORY*0..2 to discover all sub-levels, but still only return direct relationships and their nodes.
10. Provide only the Cypher query text—no comments or explanations.

Schema:
{schema}

Rules:

Every variable returned must be bound in a corresponding MATCH clause.

Return all properties for those variables in the RETURN clause.

The question is:
{question}

Examples (increasing complexity):

Question: “What is the difference in Stock in trade count for the company 133-FES from FY24 to FY23?”Cypher:

MATCH (gc:GroupCompany {name: "133-FES"})-[r:HAS_AMOUNT]->(inv:InventoryLevel4 {name: "Stock in trade"})
WHERE r.year IN ["FY23","FY24"]
RETURN gc, r, inv

Question: “Why is the Stores and Spares inventory higher for Powerol in 2024 than in 2023?”Cypher:

MATCH (inv2:InventoryLevel2 {name: "Stores and Spares"})-[:HAS_SUB_INVENTORY*0..2]->(inv4:InventoryLevel4)
MATCH (gc:GroupCompany {name: "Powerol"})-[r:HAS_AMOUNT]->(inv4)
WHERE r.year IN ["2023","2024"]
RETURN inv2, inv4, gc, r

Question: “What factors contribute to the difference in Finished Goods for SWARAJ for 2023 and 2024?”Cypher:

MATCH (inv2:InventoryLevel2 {name: "Finished Goods"})-[:HAS_SUB_INVENTORY*0..2]->(inv4:InventoryLevel4)
MATCH (gc:GroupCompany {name: "SWARAJ"})-[r:HAS_AMOUNT]->(inv4)
WHERE r.year IN ["2023","2024"]
RETURN inv2, inv4, gc, r

Question: “What is the value of WIP- Seed Potato Facility A/c for the AS sector in FY23?”Cypher:

MATCH (gc:GroupCompany {name: "AS"})-[r:HAS_AMOUNT]->(inv:InventoryLevel4 {name: "WIP- Seed Potato Facility A/c"})
WHERE r.year = "FY23"
RETURN gc, r, inv

Question: “Retrieve all first-level inventory hierarchies and amount relationships for all sub-inventories under 'Maintenance Parts' for company 'GlobalManufacturing' in 2022, including every connected GroupCompany node and their relationship properties.”Cypher:

MATCH (inv1:InventoryLevel1 {name: "Maintenance Parts"})-[:HAS_SUB_INVENTORY]->(inv2:InventoryLevel2)
MATCH (inv2)-[:HAS_SUB_INVENTORY]->(inv3:InventoryLevel3)
MATCH (inv3)-[:HAS_SUB_INVENTORY]->(inv4:InventoryLevel4)
MATCH (gc:GroupCompany {name: "GlobalManufacturing"})-[r:HAS_AMOUNT]->(inv4)
WHERE r.year = "2022"
RETURN inv1, inv2, inv3, inv4, gc, r

Question: “Show all direct connections and properties for the inventory item ‘Tools’, including any parent or child inventories and every group company amount relationship.”Cypher:

MATCH (inv2:InventoryLevel2 {name: "Tools"})-[:HAS_SUB_INVENTORY]->(inv4:InventoryLevel4)
MATCH (inv2)<-[r1:HAS_SUB_INVENTORY]-(inv1:InventoryLevel1)
MATCH (inv4)<-[r2:HAS_AMOUNT]-(gc:GroupCompany)
RETURN inv1, r1, inv2, inv4, r2, gc

"""


cypher_prompt = PromptTemplate(
    input_variables=["query"],
    template=CYPHER_GENERATION_TEMPLATE,
)



# Create the GraphCypherQAChain with the prompt template and graph
chain = GraphCypherQAChain.from_llm(
    graph=graph,
    cypher_llm=llm,
    qa_llm=llm,
    cypher_prompt=cypher_prompt,
    verbose=True,
    allow_dangerous_requests=True,
    return_intermediate_steps=True
)

## Chain without the prompt template
chain2 = GraphCypherQAChain.from_llm(
    llm, graph=graph, verbose=True, allow_dangerous_requests=True
)


In [13]:
user_query = "which inventory tools has the highest difference for AS from 2023 to 2024?"
normalized = normalize_entities(user_query)
print('INPUT QUERY NORMALIZED')
print(normalized.canonicalized_query)

query = normalized.canonicalized_query

# Run the chain with added context
result = chain2.invoke({
    "query": query,
})

print('FINAL RESULT')
print(result)

Raw LLM output: {
  "company_name": "AS",
  "inventory_type": "Inventory Tools",
  "year": "FY24",
  "canonicalized_query": "which Inventory Tools has the highest difference for AS from FY23 to FY24?"
}
INPUT QUERY NORMALIZED
which Inventory Tools has the highest difference for AS from FY23 to FY24?


> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (g:GroupCompany)-[r:HAS_AMOUNT]->(i:InventoryLevel3)
WHERE r.year IN ['FY23', 'FY24']
WITH i.name AS inventoryTool, 
     SUM(CASE WHEN r.year = 'FY24' THEN r.amount ELSE 0 END) AS amountFY24,
     SUM(CASE WHEN r.year = 'FY23' THEN r.amount ELSE 0 END) AS amountFY23
WITH inventoryTool, amountFY24, amountFY23, (amountFY24 - amountFY23) AS difference
ORDER BY difference DESC
RETURN inventoryTool, difference
LIMIT 1

Full Context:
[{'inventoryTool': 'Stock In Transit - Finised Goods', 'difference': 13978068073.599998}]

> Finished chain.
FINAL RESULT
{'query': 'which Inventory Tools has the highest difference for AS f

In [15]:
user_query = "What is the difference in Stock-in-trade count for the company 133-FES from FY24 to FY23?"

normalized = normalize_entities(user_query)
print('INPUT QUERY NORMALIZED')
print(normalized.canonicalized_query)

query = normalized.canonicalized_query

# Run the chain with added context
result = chain2.run({
    "query": query,
})

print('FINAL RESULT')
print(result)

Raw LLM output: {
  "company_name": "FES",
  "inventory_type": "Stock in Trade",
  "year": "FY24",
  "canonicalized_query": "What is the difference in Stock in Trade count for the company 133-FES from FY24 to FY23?"
}
INPUT QUERY NORMALIZED
What is the difference in Stock in Trade count for the company 133-FES from FY24 to FY23?


> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (g:GroupCompany {name: '133-FES'})-[:HAS_AMOUNT]->(i:InventoryLevel3)
WITH g, i
MATCH (g)-[:HAS_AMOUNT]->(i2:InventoryLevel4)
WHERE i.year = 'FY24' OR i.year = 'FY23'
RETURN i.year AS Year, SUM(i.amount) - SUM(i2.amount) AS StockDifference

Full Context:
[]

> Finished chain.
FINAL RESULT
I don't know the answer.


In [17]:
user_query = "Why is the stores and spares increased for Powerol from 2023 to 2024?"

normalized = normalize_entities(user_query)
print('INPUT QUERY NORMALIZED')
print(normalized.canonicalized_query)

query = normalized.canonicalized_query

# Run the chain with added context
result = chain2.run({
    "query": query,
})

print('FINAL RESULT')
print(result)

Raw LLM output: {
  "company_name": "Powerol",
  "inventory_type": "Stores and Spares",
  "year": "FY24",
  "canonicalized_query": "Why is the Stores and Spares increased for Powerol from FY23 to FY24?"
}
INPUT QUERY NORMALIZED
Why is the Stores and Spares increased for Powerol from FY23 to FY24?


> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (gc:GroupCompany {name: 'Powerol'})-[:HAS_AMOUNT]->(il3:InventoryLevel3), (gc)-[:HAS_AMOUNT]->(il4:InventoryLevel4)
WHERE il3.name = 'Stores and Spares' AND il4.name = 'Stores and Spares'
RETURN il3, il4

Full Context:
[]

> Finished chain.
FINAL RESULT
I don't know the answer.


In [19]:
user_query = "Which inventory has the highest difference and the lowest difference in count for year 2023 and 2024"

normalized = normalize_entities(user_query)
print('INPUT QUERY NORMALIZED')
print(normalized.canonicalized_query)

query = normalized.canonicalized_query

# Run the chain with added context
result = chain2.run({
    "query": query,
})

print('FINAL RESULT')
print(result)

Raw LLM output: {
  "company_name": null,
  "inventory_type": null,
  "year": "FY24",
  "canonicalized_query": "Which inventory has the highest difference and the lowest difference in count for year FY23 and FY24"
}
INPUT QUERY NORMALIZED
Which inventory has the highest difference and the lowest difference in count for year FY23 and FY24


> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (g:GroupCompany)-[r:HAS_AMOUNT]->(i:InventoryLevel3)
WHERE r.year IN ['FY23', 'FY24']
WITH i.name AS inventory, 
     SUM(CASE WHEN r.year = 'FY23' THEN r.amount ELSE 0 END) AS amountFY23,
     SUM(CASE WHEN r.year = 'FY24' THEN r.amount ELSE 0 END) AS amountFY24,
     ABS(SUM(CASE WHEN r.year = 'FY23' THEN r.amount ELSE 0 END) - 
         SUM(CASE WHEN r.year = 'FY24' THEN r.amount ELSE 0 END)) AS difference
RETURN inventory, 
       difference
ORDER BY difference DESC
LIMIT 1
UNION
MATCH (g:GroupCompany)-[r:HAS_AMOUNT]->(i:InventoryLevel3)
WHERE r.year IN ['FY23', 'FY24']
WIT

In [21]:
user_query = "which group company have lowest 'Finished products produced' count in 2023?"
normalized = normalize_entities(user_query)
print('INPUT QUERY NORMALIZED')
print(normalized.canonicalized_query)

query = normalized.canonicalized_query

# Run the chain with added context
result = chain2.run({
    "query": query,
})

print('FINAL RESULT')
print(result)

Raw LLM output: {
  "company_name": null,
  "inventory_type": "Finished Goods",
  "year": "FY23",
  "canonicalized_query": "which group company have lowest 'Finished Goods' count in FY23?"
}
INPUT QUERY NORMALIZED
which group company have lowest 'Finished Goods' count in FY23?


> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (g:GroupCompany)-[h:HAS_AMOUNT {year: 'FY23'}]->(i:InventoryLevel3 {name: 'Finished Goods'})
RETURN g.name, h.amount
ORDER BY h.amount ASC
LIMIT 1

Full Context:
[{'g.name': '139-CORP', 'h.amount': 0.0}]

> Finished chain.
FINAL RESULT
139-CORP has the lowest 'Finished Goods' count in FY23, with an amount of 0.0.
